In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from sklearn.metrics import precision_score, recall_score, mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier

# =========================
# 1️⃣ Load dataset
# =========================
df = pd.read_csv('/content/2024_S1_cdm_ccsds.csv')
df = df.infer_objects(copy=False)

cols = [
    'event_id', 'tca', 'creation_date', 'miss_distance', 'relative_speed',
    'relative_position_r', 'relative_position_t', 'relative_position_n',
    'relative_velocity_r', 'relative_velocity_t', 'relative_velocity_n',
    'object1_area_pc', 'object1_area_pc_max', 'object1_mass',
    'object2_area_pc', 'object2_area_pc_max', 'object2_mass',
    'collision_probability', 'collision_max_probability'
]
df = df[cols].copy()

# =========================
# 2️⃣ Preprocessing & Feature Engineering
# =========================
date_cols = ['tca','creation_date']
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')

df['time_to_tca'] = (df['tca'] - df['creation_date']).dt.total_seconds() / (3600*24)
df['t_cd_area_over_mass'] = df['object1_area_pc'] / df['object1_mass']
df['t_cr_area_over_mass'] = df['object1_area_pc_max'] / df['object1_mass']
df['c_cd_area_over_mass'] = df['object2_area_pc'] / df['object2_mass']
df['c_cr_area_over_mass'] = df['object2_area_pc_max'] / df['object2_mass']

df = df.drop(date_cols, axis=1)

X = df.drop(['collision_probability'], axis=1)
y = np.log10(df['collision_probability'].clip(lower=1e-15))

X = X.select_dtypes(include=[np.number])
X = X.dropna(axis=1, how='all')

imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)

# =========================
# Event-aware 80/20 split
# =========================
unique_events = df['event_id'].dropna().unique()
np.random.seed(42)
np.random.shuffle(unique_events)
n = len(unique_events)
train_e = unique_events[:int(0.8*n)]
test_e  = unique_events[int(0.8*n):]

def split_by_event(X, y, event_series, train_e, test_e):
    X_train = X[event_series.isin(train_e)]
    y_train = y[event_series.isin(train_e)]
    X_test  = X[event_series.isin(test_e)]
    y_test  = y[event_series.isin(test_e)]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_by_event(
    X_scaled, y, df['event_id'], train_e, test_e
)

risk_thr = -6
y_class_train = (y_train >= risk_thr).astype(int)
y_class_test  = (y_test >= risk_thr).astype(int)

# =========================
# CatBoost Regression
# =========================
cat_params = {
    'iterations': 1500,
    'learning_rate': 0.03,
    'depth': 8,
    'l2_leaf_reg': 3,
    'random_seed': 42,
    'loss_function': 'RMSE',
    'verbose': 0,
}
reg_model = CatBoostRegressor(**cat_params)
reg_model.fit(X_train, y_train)
y_pred_reg = reg_model.predict(X_test)

epsilon = 0.001
y_pred_reg_clipped = y_pred_reg.copy()
y_pred_reg_clipped[y_pred_reg_clipped < risk_thr] = risk_thr - epsilon

# =========================
# 5XGBoost Classification
# =========================
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0]
}

clf_model = GridSearchCV(xgb_clf, param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=0)
clf_model.fit(X_train, y_class_train)
y_pred_clf_class = clf_model.predict(X_test)

# Convert class predictions to risk values for L metric
y_pred_clf_risk = y_pred_clf_class * (risk_thr + epsilon)

# =========================
# 6Hybrid: Regression + Classification
# =========================
hybrid_prob = 0.7 * y_pred_reg + 0.3 * clf_model.predict_proba(X_test)[:,1]
hybrid_prob_clipped = hybrid_prob.copy()
hybrid_prob_clipped[hybrid_prob_clipped < risk_thr] = risk_thr - epsilon
hybrid_class = (hybrid_prob_clipped >= risk_thr).astype(int)

# =========================
# LRP baseline
# =========================
r_minus2 = np.log10(df.loc[df['event_id'].isin(test_e), 'collision_max_probability'].clip(lower=1e-15))
y_LRP = r_minus2.clip(lower=risk_thr - epsilon)

# =========================
# Metric calculation
# =========================
def compute_metrics(y_true_log, y_pred_log, threshold=-6, beta=2):
    y_true_class = (y_true_log >= threshold).astype(int)
    y_pred_class = (y_pred_log >= threshold).astype(int)
    p = precision_score(y_true_class, y_pred_class, zero_division=0)
    r = recall_score(y_true_class, y_pred_class, zero_division=0)
    F2 = (1 + beta**2) * (p * r) / (beta**2 * p + r + 1e-10)
    hr_idx = y_true_class == 1
    N_star = hr_idx.sum()
    N = len(y_true_log)
    MSEHR = (N_star / N) * mean_squared_error(y_true_log[hr_idx], y_pred_log[hr_idx]) if N_star > 0 else 0
    L = F2 * MSEHR
    return F2, MSEHR, L

F2_reg, MSEHR_reg, L_reg = compute_metrics(y_test.values, y_pred_reg_clipped)
F2_clf, MSEHR_clf, L_clf = compute_metrics(y_test.values, y_pred_clf_risk + risk_thr - epsilon)
F2_hybrid, MSEHR_hybrid, L_hybrid = compute_metrics(y_test.values, hybrid_prob_clipped)
F2_LRP, MSEHR_LRP, L_LRP = compute_metrics(y_test.values, y_LRP.values)

# =========================
# 9Standard metrics
# =========================
rmse_reg = np.sqrt(mean_squared_error(y_test, y_pred_reg))
mae_reg = mean_absolute_error(y_test, y_pred_reg)
r2_reg = r2_score(y_test, y_pred_reg)

acc_clf = accuracy_score(y_class_test, y_pred_clf_class)
f1_clf = f1_score(y_class_test, y_pred_clf_class, average='macro')

acc_hybrid = accuracy_score(y_class_test, hybrid_class)
f1_hybrid = f1_score(y_class_test, hybrid_class, average='macro')

# =========================
# Combined results table
# =========================
results_summary = pd.DataFrame({
    'Model': [
        'CatBoost Regression',
        'XGBoost Classifier',
        'Hybrid: CatBoost + XGBoost',
        'LRP Baseline'
    ],
    'RMSE': [rmse_reg, None, None, None],
    'MAE': [mae_reg, None, None, None],
    'R2': [r2_reg, None, None, None],
    'Accuracy': [None, acc_clf, acc_hybrid, None],
    'F1_macro': [None, f1_clf, f1_hybrid, None],
    'F2_score': [F2_reg, F2_clf, F2_hybrid, F2_LRP],
    'MSEHR': [MSEHR_reg, MSEHR_clf, MSEHR_hybrid, MSEHR_LRP],
    'L_metric': [L_reg, L_clf, L_hybrid, L_LRP]
})

results_summary = results_summary.sort_values(by='F2_score', ascending=False)

# =========================
# Visualization
# =========================
sns.set(style="whitegrid", font_scale=1.2)
fig, axes = plt.subplots(2,2, figsize=(15,10))

sns.barplot(x='Model', y='RMSE', data=results_summary, ax=axes[0,0], palette='viridis')
axes[0,0].set_title('RMSE (Regression)')
axes[0,0].set_ylabel('RMSE')

sns.barplot(x='Model', y='Accuracy', data=results_summary, ax=axes[0,1], palette='magma')
axes[0,1].set_title('Accuracy (Classification)')

sns.barplot(x='Model', y='F2_score', data=results_summary, ax=axes[1,0], palette='coolwarm')
axes[1,0].set_title('F2 Score')

sns.barplot(x='Model', y='L_metric', data=results_summary, ax=axes[1,1], palette='cividis')
axes[1,1].set_title('L Metric')

plt.tight_layout()
plt.show()

print("\n🔹 Final Combined Results Summary:")
print(results_summary)
print("\nBest XGBoost Parameters:", clf_model.best_params_)
